In [1]:
from utils.observation_utils import function_wrapper, lower_linalg_to_loops
from utils.transform_utils import evaluate_code_with_timeout
from random import randint, choice, shuffle
from tqdm import tqdm
import json, re, multiprocessing, os
from copy import copy

from data_generation import get_nested_loops_data

In [2]:
code = """
#map = affine_map<(d0, d1, d2) -> (d0, d1, d2)>
#map1 = affine_map<(d0) -> (d0 floordiv 49)>
#map2 = affine_map<(d0) -> (d0 mod 49)>
#map3 = affine_map<(d0) -> ((d0 mod 49) floordiv 7)>
#map4 = affine_map<(d0) -> (d0 mod 7)>
#map5 = affine_map<(d0) -> (d0 floordiv 112)>
#map6 = affine_map<(d0) -> (d0 mod 112)>
#map7 = affine_map<(d0, d1) -> ((d0 floordiv 112) * 2 + (d1 mod 49) floordiv 7)>
#map8 = affine_map<(d0, d1) -> (d0 * 2 + d1 - (d0 floordiv 112) * 224 - (d1 floordiv 7) * 7)>
#map9 = affine_map<(d0, d1, d2, d3) -> (d1, d3)>
#map10 = affine_map<(d0, d1, d2, d3) -> (d0, d3, d2)>
#map11 = affine_map<(d0, d1, d2, d3) -> (d0, d1, d2)>
  module attributes {torch.debug_module_name = "Net"} {
    func.func private @nanoTime() -> i64 attributes {llvm.emit_c_interface}
    func.func private @printFlops(f64)
    func.func private @printI64(i64)
    func.func private @printNewline()
    func.func private @printMemrefF32(tensor<*xf32>)
    func.func @matmul() -> tensor<32x64x112x112xf32> {
      %cst = arith.constant 2.000000e+00 : f32
      %cst_0 = arith.constant 0.000000e+00 : f32
      %0 = bufferization.alloc_tensor() : tensor<32x3x230x230xf32>
      %1 = linalg.fill ins(%cst : f32) outs(%0 : tensor<32x3x230x230xf32>) -> tensor<32x3x230x230xf32>
      %2 = bufferization.alloc_tensor() : tensor<64x3x7x7xf32>
      %3 = linalg.fill ins(%cst : f32) outs(%2 : tensor<64x3x7x7xf32>) -> tensor<64x3x7x7xf32>
      %4 = bufferization.alloc_tensor() : tensor<32x64x112x112xf32>
      %5 = linalg.fill ins(%cst : f32) outs(%4 : tensor<32x64x112x112xf32>) -> tensor<32x64x112x112xf32>
      %6 = call @nanoTime() : () -> i64
      %collapsed = tensor.collapse_shape %3 [[0], [1, 2, 3]] : tensor<64x3x7x7xf32> into tensor<64x147xf32>
      %collapsed_1 = tensor.collapse_shape %5 [[0], [1], [2, 3]] : tensor<32x64x112x112xf32> into tensor<32x64x12544xf32>
      %7 = tensor.empty() : tensor<32x147x12544xf32>
      %8 = linalg.generic {indexing_maps = [#map], iterator_types = ["parallel", "parallel", "parallel"]} outs(%7 : tensor<32x147x12544xf32>) attrs =  {tmp} {
      ^bb0(%out: f32):
        %13 = linalg.index 0 : index
        %14 = linalg.index 1 : index
        %15 = linalg.index 2 : index
        %c3 = arith.constant 3 : index
        %c7 = arith.constant 7 : index
        %c7_2 = arith.constant 7 : index
        %c49 = arith.constant 49 : index
        %16 = affine.apply #map1(%14)
        %17 = affine.apply #map2(%14)
        %18 = affine.apply #map3(%14)
        %19 = affine.apply #map4(%14)
        %c112 = arith.constant 112 : index
        %c112_3 = arith.constant 112 : index
        %20 = affine.apply #map5(%15)
        %21 = affine.apply #map6(%15)
        %22 = affine.apply #map7(%15, %14)
        %23 = affine.apply #map8(%15, %14)
        %extracted = tensor.extract %1[%13, %16, %22, %23] : tensor<32x3x230x230xf32>
        linalg.yield %extracted : f32
      } -> tensor<32x147x12544xf32>
      %9 = linalg.generic {indexing_maps = [#map9, #map10, #map11], iterator_types = ["parallel", "parallel", "parallel", "reduction"]} ins(%collapsed, %8 : tensor<64x147xf32>, tensor<32x147x12544xf32>) outs(%collapsed_1 : tensor<32x64x12544xf32>) attrs =  {tag = "operation_1"} {
      ^bb0(%in: f32, %in_2: f32, %out: f32):
        %13 = arith.mulf %in, %in_2 : f32
        %14 = arith.addf %13, %out : f32
        linalg.yield %14 : f32
      } -> tensor<32x64x12544xf32>
      %expanded = tensor.expand_shape %9 [[0], [1], [2, 3]] : tensor<32x64x12544xf32> into tensor<32x64x112x112xf32>
      %10 = call @nanoTime() : () -> i64
      %11 = arith.subi %10, %6 : i64
      %12 = arith.uitofp %11 : i64 to f64
      call @printI64(%11) : (i64) -> ()
      call @printNewline() : () -> ()
      return %expanded : tensor<32x64x112x112xf32>
    }
    func.func @main() {
      %c1 = arith.constant 1 : index
      %c0 = arith.constant 0 : index
      %c2 = arith.constant 2 : index
      scf.for %arg0 = %c0 to %c2 step %c1 {
        %0 = func.call @matmul() : () -> tensor<32x64x112x112xf32>
      }
      return
    }
  }
"""

In [3]:
loops = lower_linalg_to_loops(code)
loops = get_nested_loops_data(loops)

In [4]:
print(loops)
loops

{'nested_loops': [('%arg0', 0, 64, 1), ('%arg1', 0, 147, 1), ('%arg0', 0, 32, 1), ('%arg1', 0, 64, 1), ('%arg2', 0, 12544, 1), ('%arg0', 0, 32, 1), ('%arg1', 0, 64, 1), ('%arg2', 0, 12544, 1), ('%arg3', 0, 147, 1)], 'op_count': {'+': 1, '-': 0, '*': 1, '/': 0, 'exp': 0}, 'load_data': [['%arg1', '%arg3'], ['%arg0', '%arg1', '%arg2']], 'store_data': []}


{'nested_loops': [('%arg0', 0, 64, 1),
  ('%arg1', 0, 147, 1),
  ('%arg0', 0, 32, 1),
  ('%arg1', 0, 64, 1),
  ('%arg2', 0, 12544, 1),
  ('%arg0', 0, 32, 1),
  ('%arg1', 0, 64, 1),
  ('%arg2', 0, 12544, 1),
  ('%arg3', 0, 147, 1)],
 'op_count': {'+': 1, '-': 0, '*': 1, '/': 0, 'exp': 0},
 'load_data': [['%arg1', '%arg3'], ['%arg0', '%arg1', '%arg2']],
 'store_data': []}